In [ ]:

import serial
import time
from astropy.coordinates import SkyCoord, AltAz, EarthLocation
from astropy.time import Time
import numpy as np
import tkinter as tk
from tkinter import messagebox, ttk

# Observer's location (latitude, longitude, and elevation)
latitude = 42.3744   # Example: MIT, Boston, USA
longitude = -71.1167
elevation = 43  # Elevation in meters
location = EarthLocation(lat=latitude, lon=longitude, height=elevation)

# Establish serial connection (update with your port)
try:
    ser = serial.Serial('/dev/ttyUSB0', 9600, timeout=1)  # Adjust for your serial port
except serial.SerialException:
    ser = None
    print("Error: Could not connect to telescope.")

# Function to convert RA/DEC to Alt/Az
def ra_dec_to_alt_az(ra, dec):
    sky_coord = SkyCoord(ra=ra, dec=dec, unit=('hourangle', 'deg'))
    altaz = sky_coord.transform_to(AltAz(obstime=Time.now(), location=location))
    return altaz.alt.degree, altaz.az.degree

# Send Alt/Az coordinates to the telescope
def send_command_to_telescope(alt, az):
    if ser:
        alt_steps = int((alt / 360.0) * 65536)
        az_steps = int((az / 360.0) * 65536)
        command = f'R{az_steps:04X}{alt_steps:04X}'
        ser.write(command.encode())
        time.sleep(0.5)
    else:
        messagebox.showerror("Error", "Telescope not connected.")

# Track the selected object
def track_object(ra, dec):
    while True:
        alt, az = ra_dec_to_alt_az(ra, dec)
        status_var.set(f'Tracking - Alt: {alt:.2f}, Az: {az:.2f}')
        send_command_to_telescope(alt, az)
        time.sleep(5)  # Update every 5 seconds

# Start tracking when the button is clicked
def start_tracking():
    selected_object = object_var.get()
    ra, dec = celestial_objects[selected_object]
    track_object(ra, dec)

# Celestial objects with RA/DEC values
celestial_objects = {
    "Andromeda Galaxy": ("00h42m44.3s", "+41d16m09s"),
    "Orion Nebula": ("05h35m17.3s", "-05d23m28s"),
    "Pleiades": ("03h47m29.0s", "+24d07m00s"),
    "Polaris": ("02h31m49.09s", "+89d15m50.8s")
}

# GUI Setup using Tkinter
root = tk.Tk()
root.title("Celestron Telescope Tracker")

# Dropdown menu to select celestial object
object_var = tk.StringVar(value="Andromeda Galaxy")
ttk.Label(root, text="Select Celestial Object:").grid(row=0, column=0, padx=10, pady=5)
object_menu = ttk.Combobox(root, textvariable=object_var, values=list(celestial_objects.keys()))
object_menu.grid(row=0, column=1, padx=10, pady=5)

# Start tracking button
track_button = ttk.Button(root, text="Start Tracking", command=start_tracking)
track_button.grid(row=1, column=0, columnspan=2, pady=10)

# Status label to display tracking status
status_var = tk.StringVar(value="Ready")
status_label = ttk.Label(root, textvariable=status_var)
status_label.grid(row=2, column=0, columnspan=2, pady=5)

# Run the GUI event loop
root.mainloop()
